In [1]:
#!pip install numpy
#!pip install scipy

In [48]:
import numpy as np
from scipy.stats import ortho_group
from typing import Callable, Tuple
import sympy as sp
from scipy.optimize import minimize

class FunctionGenerator:
    def __init__(self, n_dims: int):
        self.n_dims = n_dims
        self.symbols = sp.symbols(f'x:{self.n_dims}')

    def generate_optimum(self) -> np.ndarray:
        return np.random.uniform(-10, 10, self.n_dims)

    def basic_function(self, optimum: np.ndarray) -> Tuple[sp.Expr, np.ndarray]:
        expr = sum((xi - ai)**2 for xi, ai in zip(self.symbols, optimum))
        expr = sp.expand(expr)  # Ensure the expression is expanded
        print(f"Basic Function Optimum:\n{optimum}")
        print(f"Basic Function Expression (Expanded):\n{expr}")
        return expr, optimum

    def apply_rotation(self, expr: sp.Expr, optimum: np.ndarray) -> Tuple[sp.Expr, np.ndarray]:
        R = ortho_group.rvs(self.n_dims)
        
        # Rotate the optimum
        new_optimum = R @ optimum
        
        # Create expressions for the rotated coordinates
        rotated_coords = R.T @ sp.Matrix(self.symbols)
        
        # Substitute the rotated coordinates into the original expression
        subs_dict = {self.symbols[i]: rotated_coords[i] for i in range(self.n_dims)}
        rotated_expr = expr.subs(subs_dict)
        
        # Expand the rotated expression
        rotated_expr = sp.expand(rotated_expr)
        
        print(f"Rotation Matrix R:\n{R}")
        print(f"Original Optimum:\n{optimum}")
        print(f"New Optimum after Rotation:\n{new_optimum}")
        print(f"Rotated Expression:\n{rotated_expr}")
        
        # Verify that the function value at the new optimum matches the original
        original_value = expr.subs(dict(zip(self.symbols, optimum))).evalf()
        rotated_value = rotated_expr.subs(dict(zip(self.symbols, new_optimum))).evalf()
        print(f"Original function value at original optimum: {original_value}")
        print(f"Rotated function value at new optimum: {rotated_value}")
        
        return rotated_expr, np.array(new_optimum).astype(np.float64)

    def generate_function(self) -> Tuple[Callable, np.ndarray, sp.Expr]:
        optimum = self.generate_optimum()
        expr, optimum = self.basic_function(optimum)
        expr, optimum = self.apply_rotation(expr, optimum)
        
        func = lambda x: float(expr.subs(dict(zip(self.symbols, x))).evalf())
        return func, optimum, expr

def try_sympy_solve(expr, symbols):
    try:
        solution = sp.solve(sp.diff(expr, symbols), symbols)
        return solution
    except Exception:
        return None

def optimize_function(f, initial_guess, bounds):
    result = minimize(f, initial_guess, method='L-BFGS-B', bounds=bounds)
    return result.x, result.fun

def generate_dataset(n_problems, n_dims):
    generator = FunctionGenerator(n_dims)
    dataset = []
    for _ in range(n_problems):
        f, optimum, expr = generator.generate_function()
        dataset.append((f, optimum, expr))
    return dataset

if __name__ == "__main__":
    np.random.seed()  # Ensures we get different random numbers each time
    generator = FunctionGenerator(n_dims=2)
    bounds = [(-20, 20)] * generator.n_dims
    
    # Generate 3 problems in 2 dimensions
    print("\nGenerating a small dataset:")
    dataset = generate_dataset(3, 2)
    for i, (f, optimum, expr) in enumerate(dataset):
        print(f"\nProblem {i+1}:")
        print(f"Optimum: {optimum}")
        print(f"Function definition:\n{expr}")
        try:
            value_at_optimum = float(f(optimum))  # Ensure numerical value
            print(f"Function value at optimum: {value_at_optimum:.6f}")
        except Exception as e:
            print(f"Error evaluating function at optimum: {e}")
        
        sympy_solution = try_sympy_solve(expr, generator.symbols)
        if sympy_solution:
            print("SymPy found an analytical solution:")
            print(sympy_solution)
        else:
            print("SymPy couldn't find an analytical solution.")
        
        # Optimization attempt
        initial_guess = optimum + np.random.normal(0, 1, generator.n_dims)
        found_optimum, found_value = optimize_function(f, initial_guess, bounds)
        print(f"Optimization result:")
        print(f"Found optimum: {np.round(found_optimum, 4)}")
        print(f"Found value: {found_value:.6f}")
        print(f"Distance from true optimum: {np.linalg.norm(found_optimum - optimum):.6f}")



Generating a small dataset:
Basic Function Optimum:
[5.1896507  2.07868638]
Basic Function Expression (Expanded):
1.0*x0**2 - 10.3793014006195*x0 + 1.0*x1**2 - 4.1573727514222*x1 + 31.2534114397925
Rotation Matrix R:
[[-0.35045962 -0.93657784]
 [ 0.93657784 -0.35045962]]
Original Optimum:
[5.1896507  2.07868638]
New Optimum after Rotation:
[-3.76561462  4.13201622]
Rotated Expression:
2.38427231522905*x0**2 + 1.4623671245491*x0*x1 + 16.6357246389931*x0 + 0.230558660830602*x1**2 + 4.86381755607868*x1 + 31.2534114397925
Original function value at original optimum: 1.77635683940025E-15
Rotated function value at new optimum: 3.69832603798109
Basic Function Optimum:
[4.16771393 8.37751005]
Basic Function Expression (Expanded):
1.0*x0**2 - 8.33542785866435*x0 + 1.0*x1**2 - 16.7550201026321*x1 + 87.5525140566509
Rotation Matrix R:
[[ 0.88642643 -0.46286951]
 [-0.46286951 -0.88642643]]
Original Optimum:
[4.16771393 8.37751005]
New Optimum after Rotation:
[-0.18332215 -9.35515404]
Rotated Expr

In [13]:
R = ortho_group.rvs(2)
print(R)

res = R.T @ R
# let's check if the result is the identity matrix
# for very small values, we can consider them as zeros, so we can use np.isclose
print(np.isclose(res, np.eye(2)))

[[ 0.9593495   0.28222072]
 [-0.28222072  0.9593495 ]]
[[ True  True]
 [ True  True]]
